<a href="https://colab.research.google.com/github/DeepHansda/VS-Code-Online---code-server/blob/main/vs_code_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Step 1: Install Necessary Tools
try:
    !pip install pyngrok
    !curl -fsSL https://code-server.dev/install.sh | sh
    !mkdir -p ~/.local/lib ~/.local/bin
    !curl -fL https://github.com/coder/code-server/releases/download/v$VERSION/code-server-$VERSION-linux-amd64.tar.gz \
      | tar -C ~/.local/lib -xz
    !mv ~/.local/lib/code-server-$VERSION-linux-amd64 ~/.local/lib/code-server-$VERSION
    !ln -s ~/.local/lib/code-server-$VERSION/bin/code-server ~/.local/bin/code-server
    !PATH="~/.local/bin:$PATH"
except Exception as e:
    print("Error installing tools:", e)

In [ ]:
import json
from threading import Timer
from queue import Queue
from pyngrok import ngrok

In [ ]:

# Step 2: Detect Environment and Retrieve Tokens
try:
    env = %pwd
    ngrok_token = None
    ngrok_domain = None
    if "/content" in env:  # Google Colab
        from google.colab import userdata
        ngrok_token = userdata.get('NGROK_TOKEN')
        ngrok_domain = userdata.get('NGROK_DOMAIN')
    elif "/kaggle" in env:  # Kaggle
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        ngrok_token = user_secrets.get_secret("NGROK_TOKEN")
        ngrok_domain = user_secrets.get_secret("NGROK_DOMAIN")

    if not ngrok_token:
        raise ValueError("Ngrok token is missing.")
except Exception as e:
    print("Error detecting environment or retrieving tokens:", e)

In [ ]:
# Step 3: Define Function to Set Up Ngrok Tunnel
def ngrok_tunnel(port, queue, auth_token):
    try:
        ngrok.set_auth_token(auth_token)
        url = ngrok.connect(port, domain=ngrok_domain)
        queue.put(url)
    except Exception as e:
        print("Error starting ngrok tunnel:", e)

In [ ]:

# Step 4: Start the Ngrok Tunnel
try:
    ngrok_output_queue = Queue()
    ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
    ngrok_thread.start()
    ngrok_thread.join()
    print("Ngrok URL:", ngrok_output_queue.get())
except Exception as e:
    print("Error starting ngrok thread:", e)

In [ ]:

# Step 5: Print Tunnels (If Any)
try:
    tunnels = ngrok.get_tunnels()
    print("Active tunnels:", tunnels)
except Exception as e:
    print("Error retrieving tunnels:", e)

# Step 6: Start Code Server
try:
    !code-server --auth none --port 8188
except Exception as e:
    print("Error starting code-server:", e)